# Saving and loading in DeepSensor

This notebook demonstrates how to use saving and loading functionality in DeepSensor. The three key DeepSensor classes: the `DataProcessor`, `TaskLoader`, and `ConvNP`, can all be saved and loaded. Internally, config files are used to track the state of each class, and these config files are saved alongside the model weights (for the `ConvNP`).

In [1]:
import logging
logging.captureWarnings(True)

import os

import deepsensor.torch
from deepsensor.data.loader import TaskLoader
from deepsensor.data.processor import DataProcessor
from deepsensor.model.convnp import ConvNP

import xarray as xr
import pandas as pd

In [2]:
folder = f"tmp_{pd.Timestamp.now().strftime('%Y%m%d_%H%M%S')}"
os.makedirs(folder, exist_ok=True)

## Load raw data

In [3]:
ds_raw = xr.tutorial.open_dataset("air_temperature")

## Normalise/preprocess data

In [4]:
data_processor = DataProcessor(x1_name="lat", x2_name="lon", verbose=True)
ds = data_processor(ds_raw)

Normalisation params for air not computed. Computing now... Done. air mean_std params={'mean': 281.255126953125, 'std': 16.320409774780273}
Inferring x1_map=(15.0, 75.0) and x2_map=(200.0, 330.0) from data min/max


## NEW: Save normalised data for TaskLoader

In [5]:
t2m_fpath = f"{folder}/air_temperature_normalised.nc"
ds.to_netcdf(t2m_fpath)

## Set up TaskLoader

We must instantiate the `TaskLoader` with *paths* rather than the xarray object, so that the `TaskLoader` can be saved and loaded.

In [6]:
task_loader = TaskLoader(context=t2m_fpath, target=t2m_fpath)

## Set up ConvNP model

In [7]:
model = ConvNP(data_processor, task_loader, unet_channels=(32,)*3, verbose=False)

In [8]:
test_task = task_loader("2014-12-31", 40, datewise_deterministic=True)
mean_ds_before, std_ds_before = model.predict(test_task, X_t=ds_raw)

## Save the DataProcessor, TaskLoader, and ConvNP

In [9]:
data_processor.save(folder)
task_loader.save(folder)
model.save(folder)

## Load the DataProcessor, TaskLoader, and ConvNP

In [10]:
data_processor_loaded = DataProcessor(folder)
task_loader_loaded = TaskLoader(folder)
model_loaded = ConvNP(data_processor, task_loader, folder)

In [11]:
test_task = task_loader_loaded("2014-12-31", 40, datewise_deterministic=True)
mean_ds_loaded, std_ds_loaded = model_loaded.predict(test_task, X_t=ds_raw)

In [12]:
xr.testing.assert_allclose(mean_ds_before, mean_ds_loaded)
print("Means match")

Means match


In [13]:
xr.testing.assert_allclose(std_ds_before, std_ds_loaded)
print("Standard deviations match")

Standard deviations match


## Delete temporary folder

In [14]:
import shutil
shutil.rmtree(folder)